In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy import sparse

import time 

import multiprocessing
from itertools import repeat


import matplotlib.pyplot as plt
%matplotlib inline


from pyomo.environ import *

### This notebook solves problem (11) in the manuscript to identify the EV charging demand that can be supported
- $\omega$ represents the percentage of charging demand that can be supported (shape: N_e_total).
- We totally have 21760 trip chains. Due to the memory limitation, we divide these trip chains into 22 groups with 1000 chains in each.
- It calculates the value of $\omega$ under spatial distribution of EV hosting capacity (Figs. 6(d)-(f))

In [2]:
N_n = 7
N_t = 96
feasibleTrip = np.load('Parameters/Chicago2017_RealCapacity/pubCharger/feasible_chain.npy')
N_e_total = feasibleTrip.shape[0]

q = sparse.load_npz('Parameters/Chicago2017_RealCapacity/pubCharger/q_rec.npz').toarray()[0][feasibleTrip]

E_init = 0.5
E_terminal = 0.5
L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1
demand_cum = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/demand_cum.npz").toarray()[feasibleTrip]

N_e = 1000
groupNum = N_e_total // N_e
N_e_last = N_e_total - N_e * groupNum


Location = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/Location.npz")[feasibleTrip]


In [4]:
mapping = pd.read_csv('TripChains/county_Mapping.csv', index_col = 0)
mappings = {}
for i in range(len(mapping)):
    mappings[mapping['nodeIndex'][i]] = mapping['countyIndex'][i]
countyIndices = mapping['countyIndex'].unique()
countyNodes = {}
for n in range(len(countyIndices)):
    countyNodes[countyIndices[n]] = n

In [5]:
boolIndex = {}
for countyInd in countyIndices:
    boolIndex[countyInd] = np.zeros((N_e_total, N_t))
    for t in range(N_t):
        boolIndex[countyInd][:, t] = Location[:, t].toarray().T[0] == countyInd


def getCountyInd(x):
    if x == -1:
        return -1
    else:
        return countyNodes[x]

vectorized_f = np.vectorize(getCountyInd)
location_countyID = vectorized_f(Location.toarray())
    

In [6]:
'''problem for each group'''

model = ConcreteModel()

# Variables

model.y_e = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.E_e_omega = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0) # stored energy
model.omega = Var(range(N_e), domain=NonNegativeReals, bounds=(0, 1), initialize=1)

model.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model.x_e_UB = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) # p_max
model.demand_cum = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) # demand
model.q_e = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model.coe_county0 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) # pene * q_e * boolIndex[countyInd]
model.coe_county1 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county2 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county3 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county4 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county5 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county6 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)



# Constraints for individual trip chain
model.c1 = ConstraintList()
for e in range(N_e):
    for t in range(N_t):
        model.c1.add(expr=model.y_e[e, t] <= model.omega[e] * model.x_e_UB[e, t])
        model.c1.add(expr=E_init * model.omega[e] - model.demand_cum[e, t] * model.omega[e] + 
                     sum(L[t, tt] * model.y_e[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_omega[e, t])
        model.c1.add(expr=model.E_e_omega[e, t] >= 0)
        model.c1.add(expr=model.E_e_omega[e, t] <= model.omega[e])
    model.c1.add(expr=model.E_e_omega[e, N_t-1] >= model.omega[e] * E_terminal)

# Constraints for aggregate charging power capacity
model.c2 = ConstraintList()
for t in range(N_t):
    model.c2.add(sum(model.coe_county0[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[0])
    model.c2.add(sum(model.coe_county1[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[1])
    model.c2.add(sum(model.coe_county2[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[2])
    model.c2.add(sum(model.coe_county3[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[3])
    model.c2.add(sum(model.coe_county4[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[4])
    model.c2.add(sum(model.coe_county5[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[5])
    model.c2.add(sum(model.coe_county6[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[6])

# Objective
model.obj = Objective(expr=sum(model.q_e[e] * model.omega[e] for e in range(N_e)), sense=maximize)

In [7]:
'''problem for each group'''

model_last = ConcreteModel()

# Variables

model_last.y_e = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.E_e_omega = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0) # stored energy
model_last.omega = Var(range(N_e_last), domain=NonNegativeReals, bounds=(0, 1), initialize=1)

model_last.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model_last.x_e_UB = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) # p_max
model_last.demand_cum = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) # demand
model_last.q_e = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model_last.coe_county0 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) # pene * q_e * boolIndex[countyInd]
model_last.coe_county1 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county2 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county3 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county4 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county5 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county6 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

# Constraints for individual trip chain
model_last.c1 = ConstraintList()
for e in range(N_e_last):
    for t in range(N_t):
        model_last.c1.add(expr=model_last.y_e[e, t] <= model_last.omega[e] * model_last.x_e_UB[e, t])
        model_last.c1.add(expr=E_init * model_last.omega[e] - model_last.demand_cum[e, t] * model_last.omega[e] + 
                     sum(L[t, tt] * model_last.y_e[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_omega[e, t])
        model_last.c1.add(expr=model_last.E_e_omega[e, t] >= 0)
        model_last.c1.add(expr=model_last.E_e_omega[e, t] <= model_last.omega[e])
    model_last.c1.add(expr=model_last.E_e_omega[e, N_t-1] >= model_last.omega[e] * E_terminal)

# Constraints for aggregate charging power capacity
model_last.c2 = ConstraintList()
for t in range(N_t):
    model_last.c2.add(sum(model_last.coe_county0[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[0])
    model_last.c2.add(sum(model_last.coe_county1[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[1])
    model_last.c2.add(sum(model_last.coe_county2[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[2])
    model_last.c2.add(sum(model_last.coe_county3[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[3])
    model_last.c2.add(sum(model_last.coe_county4[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[4])
    model_last.c2.add(sum(model_last.coe_county5[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[5])
    model_last.c2.add(sum(model_last.coe_county6[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[6])

# Objective
model_last.obj = Objective(expr=sum(model_last.q_e[e] * model_last.omega[e] for e in range(N_e_last)), sense=maximize)



In [ ]:
peneList = [0.5]

ratio_pubCharger_list = [0.4]
ratioNum = len(ratio_pubCharger_list)

scale = 6
capacity_county = pd.read_csv('geo/capacity_county_ordered.csv', index_col = 0)['EV_LOAD_CAPACITY_KW'].values/1000 / 56.5 * 1000
capacity_total = capacity_county.sum() * scale
peneNum = 10

proportionNum = 9
proportion_cook = np.linspace(0.1, 0.9, proportionNum)
share_restCounties = capacity_county[1:]/capacity_county[1:].sum()

capacity_distribution = np.zeros((proportionNum, N_n))
for p in range(proportionNum):
    capacity_distribution[p, 0] = capacity_total * proportion_cook[p]
    capacity_distribution[p, 1:] = capacity_total * share_restCounties * (1 - proportion_cook[p])

In [ ]:
omega_rec = {}

solver = SolverFactory('gurobi')

groupShare_matrix = sparse.load_npz(
                f"Parameters/Chicago2017_RealCapacity/pubCharger/groupShare/group.npz").toarray()
for i in [0]:
    omega_rec[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list[0]:
        flag_group = np.zeros(groupNum+1)
        omega_rec[i][ratio_pubCharger] = np.zeros((proportionNum, N_e_total))
        x_UB_value = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()[feasibleTrip]
        for p in range(proportionNum):
            capacity_value = capacity_distribution[p]
            print (f"start: ratio_pubCharger={ratio_pubCharger}, proportion={proportion_cook[p]}, EV_pene={peneList[i]}")
            pene_value = peneList[i]
            groupShare = groupShare_matrix[i].reshape(groupNum+1, N_n)
            model.pene_para = pene_value
            start_all = time.time()
            for g in range(groupNum):


                '''parameter seeting for each group'''
                for n in range(N_n):
                    model.ell_capacity[n] = capacity_value[n] * groupShare[g, n]

                for e in range(N_e):
                    model.q_e[e] = q[g * N_e + e]
                    for t in range(N_t):
                        model.x_e_UB[e, t] = x_UB_value[g * N_e + e, t]
                        model.demand_cum[e, t] = demand_cum[g * N_e + e, t]
                        model.coe_county0[e, t] = pene_value * q[g * N_e + e] * boolIndex[31][g * N_e + e, t]
                        model.coe_county1[e, t] = pene_value * q[g * N_e + e] * boolIndex[43][g * N_e + e, t]
                        model.coe_county2[e, t] = pene_value * q[g * N_e + e] * boolIndex[89][g * N_e + e, t]
                        model.coe_county3[e, t] = pene_value * q[g * N_e + e] * boolIndex[97][g * N_e + e, t]
                        model.coe_county4[e, t] = pene_value * q[g * N_e + e] * boolIndex[111][g * N_e + e, t]
                        model.coe_county5[e, t] = pene_value * q[g * N_e + e] * boolIndex[197][g * N_e + e, t]
                        model.coe_county6[e, t] = pene_value * q[g * N_e + e] * boolIndex[93][g * N_e + e, t]

                '''solve the problem for each group'''
                start = time.time()
                results = solver.solve(model)

                if str(results['Solver'][0]['Termination condition']) == 'optimal':
                    omega_temp = np.array(model.omega[:]())
                    omega_rec[i][ratio_pubCharger][p, g * N_e: (g+1) * N_e] = omega_temp
                    print (f"group={g}, obj={omega_temp.mean()*100:.2f}%")
                    if omega_temp.mean() == 1:
                        flag_group[g] = 1
                else:
                    print (f"group={g}, infeasible")


            '''solve the problem for the last group'''

            model_last.pene_para = pene_value
            for n in range(N_n):
                model_last.ell_capacity[n] = capacity_value[n] * groupShare[groupNum, n]

            for e in range(N_e_last):
                model_last.q_e[e] = q[groupNum * N_e + e]
                for t in range(N_t):
                    model_last.x_e_UB[e, t] = x_UB_value[groupNum * N_e + e, t]
                    model_last.demand_cum[e, t] = demand_cum[groupNum * N_e + e, t]
                    model_last.coe_county0[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[31][groupNum * N_e + e, t]
                    model_last.coe_county1[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[43][groupNum * N_e + e, t]
                    model_last.coe_county2[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[89][groupNum * N_e + e, t]
                    model_last.coe_county3[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[97][groupNum * N_e + e, t]
                    model_last.coe_county4[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[111][groupNum * N_e + e, t]
                    model_last.coe_county5[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[197][groupNum * N_e + e, t]
                    model_last.coe_county6[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[93][groupNum * N_e + e, t]

            results_last = solver.solve(model_last)
            if str(results_last['Solver'][0]['Termination condition']) == 'optimal':
                omega_temp = np.array(model_last.omega[:]())
                omega_rec[i][ratio_pubCharger][p, groupNum * N_e: N_e_total] = omega_temp
                print (f"group={g}, percentage={omega_rec[i][ratio_pubCharger][p].mean()*100:.2f}%, time={time.time()-start_all}")
                if omega_temp.mean() == 1:
                    flag_group[groupNum] = 1
            else:
                print (f"group={g}, infeasible")

            # break # only one proportion
        break # only one ratio
    break # only one pene    
                

start: ratio_pubCharger=0.4, proportion=0.1, EV_pene=0.5
group=0, obj=100.00%
group=1, obj=100.00%
group=2, obj=100.00%
group=3, obj=100.00%
group=4, obj=99.53%
group=5, obj=100.00%
group=6, obj=100.00%
group=7, obj=100.00%
group=8, obj=100.00%
group=9, obj=100.00%
group=10, obj=100.00%
group=11, obj=92.46%
group=12, obj=91.97%
group=13, obj=90.77%
group=14, obj=92.33%
group=15, obj=93.68%
group=16, obj=95.42%
group=17, obj=92.44%
group=18, obj=91.62%
group=19, obj=93.00%
group=20, obj=94.68%
group=20, percentage=96.44%, time=275.84698486328125
start: ratio_pubCharger=0.4, proportion=0.2, EV_pene=0.5
group=0, obj=100.00%
group=1, obj=100.00%
group=2, obj=100.00%
group=3, obj=100.00%
group=4, obj=100.00%
group=5, obj=100.00%
group=6, obj=100.00%
group=7, obj=100.00%
group=8, obj=100.00%
group=9, obj=100.00%
group=10, obj=100.00%
group=11, obj=100.00%
group=12, obj=100.00%
group=13, obj=100.00%
group=14, obj=100.00%
group=15, obj=100.00%
group=16, obj=100.00%
group=17, obj=100.00%
group=

In [19]:
filePath = 'Results/Chicago2017_RealCapacity/omega_spatial'
omega_sparse = sparse.csr_matrix(omega_rec[i][ratio_pubCharger])
sparse.save_npz(f"{filePath}/omega_pene={peneList[i]:.1f}_ratio={ratio_pubCharger:.1f}.npz", omega_sparse)